In [1]:
# Example: uploading title/abstract/category CSV from your local machine
from google.colab import files
uploaded = files.upload()


Saving data_ass3.json to data_ass3.json


In [2]:
!pip install sentence-transformers scikit-learn joblib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
!pip install transformers


In [4]:
import pandas as pd

In [5]:
df = pd.read_json("data_ass3.json")


In [6]:
# Create main_category from first in list
df['main_category'] = df['categories'].apply(lambda x: x[0] if isinstance(x, list) else str(x).split(',')[0])

# Create text = title + summary
df['text'] = df['title'] + ' ' + df['summary']

# Check your data
df[['text', 'main_category']].head()

,text,main_category
0,Tensor-network toolbox for probing dynamics of...,hep-lat
1,Unveiling the complexity of Arnold's tongues i...,physics.optics
2,Planetesimal formation via the streaming insta...,astro-ph.EP
3,Carbon capture capacity estimation of taiga re...,physics.comp-ph
4,Theoretical determination of the ionization po...,physics.atom-ph


In [8]:
# Embed text with MiniLM
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
texts = df['text'].tolist()
labels = df['main_category'].tolist()

embeddings = model.encode(texts, show_progress_bar=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1950 [00:00<?, ?it/s]

In [9]:
# STEP 6: Train classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Split
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

# Train
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate
print(classification_report(y_test, clf.predict(X_test)))


                    precision    recall  f1-score   support

       astro-ph.CO       0.81      0.62      0.70       134
       astro-ph.EP       0.81      0.84      0.83       105
       astro-ph.GA       0.76      0.88      0.81       248
       astro-ph.HE       0.77      0.73      0.75       178
       astro-ph.IM       0.52      0.59      0.55        66
       astro-ph.SR       0.73      0.67      0.70       123
   cond-mat.dis-nn       0.67      0.09      0.16        22
 cond-mat.mes-hall       0.62      0.60      0.61       183
 cond-mat.mtrl-sci       0.68      0.79      0.73       298
    cond-mat.other       0.00      0.00      0.00         7
cond-mat.quant-gas       0.70      0.43      0.53        37
     cond-mat.soft       0.64      0.71      0.68       101
cond-mat.stat-mech       0.53      0.51      0.52        79
   cond-mat.str-el       0.58      0.58      0.58       123
 cond-mat.supr-con       0.69      0.79      0.74        63
             cs.AI       0.50      0.33

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save model and embedder
joblib.dump(clf, 'logreg_model.pkl')
model.save('minilm_model')

# Download them
from google.colab import files
files.download('logreg_model.pkl')      # ← download classifier
!zip -r minilm_model.zip minilm_model
files.download('minilm_model.zip')      # ← download embedding model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: minilm_model/ (stored 0%)
  adding: minilm_model/model.safetensors (deflated 9%)
  adding: minilm_model/vocab.txt (deflated 53%)
  adding: minilm_model/sentence_bert_config.json (deflated 4%)
  adding: minilm_model/modules.json (deflated 62%)
  adding: minilm_model/tokenizer_config.json (deflated 73%)
  adding: minilm_model/config.json (deflated 48%)
  adding: minilm_model/1_Pooling/ (stored 0%)
  adding: minilm_model/1_Pooling/config.json (deflated 57%)
  adding: minilm_model/config_sentence_transformers.json (deflated 34%)
  adding: minilm_model/tokenizer.json (deflated 71%)
  adding: minilm_model/README.md (deflated 64%)
  adding: minilm_model/2_Normalize/ (stored 0%)
  adding: minilm_model/special_tokens_map.json (deflated 80%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Logistic regression with class weight balanced

In [10]:
# Logistic Regression with Class Weights
clf_balanced = LogisticRegression(max_iter=1000, class_weight='balanced')
clf_balanced.fit(X_train, y_train)

# Evaluate
print("Logistic Regression (class_weight=balanced)")
print(classification_report(y_test, clf_balanced.predict(X_test)))

# Save and download
joblib.dump(clf_balanced, 'logreg_balanced_model.pkl')
files.download('logreg_balanced_model.pkl')


Logistic Regression (class_weight=balanced)
                    precision    recall  f1-score   support

       astro-ph.CO       0.74      0.72      0.73       134
       astro-ph.EP       0.83      0.82      0.82       105
       astro-ph.GA       0.81      0.81      0.81       248
       astro-ph.HE       0.77      0.69      0.73       178
       astro-ph.IM       0.46      0.62      0.53        66
       astro-ph.SR       0.75      0.63      0.68       123
   cond-mat.dis-nn       0.22      0.50      0.31        22
 cond-mat.mes-hall       0.65      0.55      0.59       183
 cond-mat.mtrl-sci       0.83      0.63      0.72       298
    cond-mat.other       0.00      0.00      0.00         7
cond-mat.quant-gas       0.49      0.65      0.56        37
     cond-mat.soft       0.66      0.63      0.65       101
cond-mat.stat-mech       0.49      0.53      0.51        79
   cond-mat.str-el       0.53      0.59      0.55       123
 cond-mat.supr-con       0.61      0.84      0.71      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# XGBoost


In [15]:
# Install xgboost
!pip install xgboost

# Import libraries
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
# Assuming y_train and y_test are lists of strings from the previous steps
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=len(label_encoder.classes_))


# Train the XGBoost classifier using the encoded labels
xgb_clf.fit(X_train, y_train_encoded)

# Evaluate the model using the encoded test labels

print("XGBoost Classifier")
print(classification_report(y_test_encoded, xgb_clf.predict(X_test)))

XGBoost Classifier
              precision    recall  f1-score   support

           0       0.60      0.54      0.57       134
           1       0.72      0.67      0.69       105
           2       0.71      0.71      0.71       248
           3       0.62      0.59      0.61       178
           4       0.39      0.39      0.39        66
           5       0.54      0.56      0.55       123
           6       0.33      0.14      0.19        22
           7       0.46      0.50      0.48       183
           8       0.61      0.64      0.63       298
           9       0.00      0.00      0.00         7
          10       0.55      0.30      0.39        37
          11       0.56      0.53      0.55       101
          12       0.42      0.42      0.42        79
          13       0.47      0.47      0.47       123
          14       0.66      0.62      0.64        63
          15       0.30      0.24      0.27       249
          16       0.31      0.32      0.32        31
        

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Fine tuning MiniLM


In [7]:
!pip install transformers datasets scikit-learn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
import pandas as pd
import numpy as np
import torch


In [8]:
# Encode labels to integers
le = LabelEncoder()
df['label'] = le.fit_transform(df['main_category'])

# Split into train/test
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Convert to Hugging Face Dataset
train_ds = Dataset.from_pandas(train_df[['text', 'label']])
test_ds = Dataset.from_pandas(test_df[['text', 'label']])

In [24]:
model = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)


TypeError: BertForSequenceClassification.__init__() got an unexpected keyword argument 'NUM_CLASSES'

In [10]:
num_labels = df['label'].nunique()

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)


pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"  # 👈 disables wandb
)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics
)
import os
os.environ["WANDB_DISABLED"] = "offline"

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,3.568400,2.992544,0.310226
2,2.738600,2.590189,0.382113
3,2.454600,2.472902,0.393893


TrainOutput(global_step=9360, training_loss=2.9205414079193375, metrics={'train_runtime': 1204.9543, 'train_samples_per_second': 124.267, 'train_steps_per_second': 7.768, 'total_flos': 2472432322203648.0, 'train_loss': 2.9205414079193375, 'epoch': 3.0})

In [21]:
eval_results = trainer.evaluate()
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']:.4f}")

Evaluation Accuracy: 0.3939


In [22]:
text = "A paper about neural networks for image classification"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    pred = torch.argmax(logits, dim=1).item()

print("Predicted label:", le.inverse_transform([pred])[0])



RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)